In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
import xgboost as xgb

In [23]:
 df = pd.read_csv("C:\\Users\\ASUS\\train.csv") #sesuaikan path
df.head(5000)

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0
...,...,...,...,...,...,...,...,...,...,...
1874,1,321,76.03,8,7,2,0.071651,0.236854,0.347826,0
1875,1,397,41.89,5,5,0,0.065491,0.105516,0.192308,1
1876,4,316,41.83,5,8,1,0.094937,0.132373,0.166667,0
1877,2,685,62.68,1,6,2,0.035036,0.091504,0.041667,0


inti dari nih tugas, kita ngeklasifikasi dari data. apakah per baris tuh nipu atau ga

trustLevel = level kepercayaan (kynya dari history)
totalScanTimeInSeconds = total waktu (detik) ngescan dari produk pertama s/d terakhir
grandTotal = jumlah produk yg di-scan
lineItemVoids = jumlah scan yang batal
scansWithoutRegistration = brp kali ngeaktifin scanner tanpa ngescan barang
quantityModification = ada brp produk yg kuantitasnya diubah
scannedLineItemsPerSecond =  rata2 ada brp produk yg di-scan per detik
valuePerSecond = rata2 total value (harga) produk yg di-scan per detik
lineItemVoidsPerPosition = rata2 produk batal per semua produk yang di-scan dan tidak batal
fraud = nipu/ga

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df.fraud)
Y = le.transform(df.fraud)

In [8]:
X = df.drop("fraud",axis=1)

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=123)

#simpan nama kolom untuk keperluan prediksi nanti
import pickle
with open('C:\\Users\\ASUS\\fraud_x_train_columns.pickle', 'wb') as fp:
    pickle.dump(X_train.columns, fp)

In [11]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
clf = GaussianNB()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
print("Akurasi {}".format(acc))
print(classification_report(Y_test, Y_pred))

Akurasi 0.8670212765957447
              precision    recall  f1-score   support

           0       1.00      0.86      0.93       361
           1       0.23      1.00      0.38        15

    accuracy                           0.87       376
   macro avg       0.62      0.93      0.65       376
weighted avg       0.97      0.87      0.90       376



In [12]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
print("Akurasi {}".format(acc))
print(classification_report(Y_test, Y_pred))

Akurasi 0.973404255319149
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       361
           1       0.73      0.53      0.62        15

    accuracy                           0.97       376
   macro avg       0.85      0.76      0.80       376
weighted avg       0.97      0.97      0.97       376



In [13]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=50, random_state=123)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
print("Akurasi {}".format(acc))
print(classification_report(Y_test, Y_pred))

Akurasi 0.9867021276595744
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       361
           1       1.00      0.67      0.80        15

    accuracy                           0.99       376
   macro avg       0.99      0.83      0.90       376
weighted avg       0.99      0.99      0.99       376



In [14]:
feature_importances = pd.DataFrame(clf.feature_importances_,index = X_train.columns,
columns=['importance']).sort_values('importance',ascending=False)
print(feature_importances)

                           importance
trustLevel                   0.275139
totalScanTimeInSeconds       0.153191
scannedLineItemsPerSecond    0.149126
lineItemVoidsPerPosition     0.099001
valuePerSecond               0.087869
grandTotal                   0.081010
lineItemVoids                0.072523
scansWithoutRegistration     0.056474
quantityModifications        0.025666


In [15]:
clf = xgb.XGBClassifier(objective = "multi:softprob", num_class = 4, eval_metric =
"mlogloss", max_depth = 24, gamma=0.1, subsample = 0.90,learning_rate=0.01,
n_estimators = 10, nthread=-1)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
print("Akurasi {}".format(acc))
print(classification_report(Y_test, Y_pred))

C:\Users\ASUS\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Akurasi 0.9920212765957447
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       361
           1       1.00      0.80      0.89        15

    accuracy                           0.99       376
   macro avg       1.00      0.90      0.94       376
weighted avg       0.99      0.99      0.99       376



In [16]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(Y_test, Y_pred))

[[361   0]
 [  3  12]]


In [24]:
df = df[df.fraud>0]

In [25]:
df.head(1879)

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
5,1,770,11.09,11,5,2,0.033766,0.014403,0.423077,1
37,2,1397,62.59,7,9,4,0.017895,0.044803,0.280000,1
53,1,1385,34.68,5,8,4,0.019495,0.025040,0.185185,1
91,1,996,71.94,7,0,1,0.027108,0.072229,0.259259,1
97,2,1305,87.65,7,8,4,0.021456,0.067165,0.250000,1
...,...,...,...,...,...,...,...,...,...,...
1822,2,1689,13.70,8,1,1,0.017762,0.008111,0.266667,1
1838,1,1465,58.95,10,0,2,0.017065,0.040239,0.400000,1
1852,1,801,73.63,6,1,4,0.034956,0.091923,0.214286,1
1857,1,1400,59.43,3,6,2,0.016429,0.042450,0.130435,1


In [26]:
from sklearn.model_selection import cross_val_score
clf = tree.DecisionTreeClassifier()
scores = cross_val_score(clf, X=X, y=Y, cv=3)
print(scores)


[0.96491228 0.95686901 0.96166134]


In [27]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

def classification_report_with_accuracy_score(y_true, y_pred):    
    print(classification_report(y_true, y_pred)) 
    print(confusion_matrix(y_true, y_pred))
    return accuracy_score(y_true, y_pred) 
clf = tree.DecisionTreeClassifier()
scores = cross_val_score(clf, X=X, y=Y, cv=3, 
                         scoring=make_scorer(classification_report_with_accuracy_score))
print(scores)


              precision    recall  f1-score   support

           0       0.98      0.98      0.98       592
           1       0.71      0.63      0.67        35

    accuracy                           0.96       627
   macro avg       0.84      0.81      0.82       627
weighted avg       0.96      0.96      0.96       627

[[583   9]
 [ 13  22]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       592
           1       0.68      0.74      0.70        34

    accuracy                           0.97       626
   macro avg       0.83      0.86      0.84       626
weighted avg       0.97      0.97      0.97       626

[[580  12]
 [  9  25]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       591
           1       0.69      0.51      0.59        35

    accuracy                           0.96       626
   macro avg       0.83      0.75      0.78       626
weighted avg       0.96     

In [33]:
from sklearn.externals import joblib
joblib.dump(clf, "C:\\Users\\ASUS\\dec_tree_fraud.joblib')

SyntaxError: EOL while scanning string literal (<ipython-input-33-8ea3a2d8ed90>, line 2)

In [34]:
df_train = pd.read_csv("C:\\Users\\ASUS\\train.csv")
cat_col = ["fraud"]
for col in cat_col: 
    df_train[col] = df_train[col].astype("category") 


In [37]:
import pickle
with open ('C:\\Users\\ASUS\\fraud_x_train_columns.pickle', 'rb') as fp:
    X_train_column = list(pickle.load(fp))
    
df_prediksi2 = df_prediksi2[X_train_column]  


NameError: name 'df_prediksi2' is not defined